In [1]:
import time
from selenium import webdriver
#from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import translators as ts
import json


Using Hong Kong server backend.


In [2]:
url="https://ieeexplore.ieee.org/document/9411225"
url="https://ieeexplore.ieee.org/document/7809977" #会议
url="https://ieeexplore.ieee.org/document/5165285" #TWC
url='https://ieeexplore.ieee.org/document/8524609'
html=urlopen(url)
html_byte=html.read()

driver = driver = webdriver.Chrome(executable_path=".\\chromedriver.exe",options=webdriver.ChromeOptions().add_argument("--headless"))# type: ignore
driver.get(url)
time.sleep(5)
html = driver.page_source
driver.quit()

bs=BeautifulSoup(html_byte)
bs_driver=BeautifulSoup(html,'lxml' )

C:\Users\Anable\AppData\Local\Temp\ipykernel_33988\2132066624.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=".\\chromedriver.exe")


In [44]:
f=open(".\\html\\get detail test.html","wb")
f.write(html_byte)
f.close()

In [45]:
f=open(".\\html\\get detail test2.html","w")
f.write(html)
f.close()


获取摘要和翻译

In [3]:
abstract = bs.find('meta', {'property': 'twitter:description'})['content']#.replace('n','') # type: ignore
print(abstract)

abstract_translation=ts.google(abstract,from_language="en",to_language='zh', host_url="https://translate.google.com/")
print(abstract_translation)

Chatbots appeared in large numbers at the beginning of the current decade. Interactive technology, often combined with artificial intelligence, has rapidly invaded and occupied the world of online chat. Chatbots are not just elements of virtual assistants, but are used by organizations and governments on websites, in applications, and instant messaging platforms to promote products, ideas or services. In this paper, the authors firstly present a theoretical and historical background, then discuss the issues of using chatbots as educational assistants, and finally describe the basic steps and challenges of programming a bot.
聊天机器人在当前十年开始时大量出现。交互式技术通常与人工智能相结合，已迅速入侵并占据了在线聊天的世界。聊天机器人不仅是虚拟助手的要素，而且在网站，应用程序和即时消息平台上的组织和政府使用以促进产品，想法或服务。在本文中，作者首先提出了理论和历史背景，然后讨论使用聊天机器人作为教育助手的问题，最后描述了编程机器人的基本步骤和挑战。


获取关键词

In [4]:
temp=bs.find("body",{"class","body-resp"}).find('div', id="LayoutWrapper").find('div', {"class",'col'}).find_all("script", type="text/javascript")

for script_tag in temp:
    if script_tag:
        script_text=script_tag.string
        if script_text:
            match = re.search(r'xplGlobal\.document\.metadata\s*=\s*(\{.*?\};)', script_text, re.DOTALL) #正则表达式匹配
            if match:
                metadata_text = match.group()
                print(metadata_text)
                break
        else:
            continue
    else:
        continue

keywords_string=re.search(r'"keywords":\s*\[.*?\],"', metadata_text, re.DOTALL).group()

All_Keywords_list_string=re.search(r'\[.*\]', keywords_string, re.DOTALL)

keywords_list=json.loads(All_Keywords_list_string.group())[3]['kwd']
print(keywords_list)

xplGlobal.document.metadata={"userInfo":{"customerNameRaw":"SHENZHEN UNIVERSITY","institutionName":"SHENZHEN UNIVERSITY","institute":true,"member":false,"individual":false,"guest":false,"subscribedContent":false,"fileCabinetContent":false,"fileCabinetUser":false,"institutionalFileCabinetUser":false,"products":"IEL|VDE|NOKIA BELL LABS|","instType":"Academic","userIds":[12631],"showPatentCitations":true,"showGet802Link":true,"openUrlImgLoc":"/assets/img/btn.find-in-library.png","openUrlLink":"NA","showOpenUrlLink":false,"marketingInfoCaptured":false,"tracked":false,"ringGoldId":"47890","desktop":false,"delegatedAdmin":false,"isInstitutionDashboardEnabled":false,"isInstitutionProfileEnabled":false,"isRoamingEnabled":true,"isDelegatedAdmin":false,"isMdl":false,"isCwg":false,"isIel":true,"isAcademic":true},"authors":[{"name":"Fei Wen","affiliation":["Department of Electronic Engineering, Shanghai Jiao Tong University, Shanghai, China","Air Force Engineering University, Xian, China"],"firstN

获取参考文献

In [6]:
View_Article=bs_driver.find_all("a",{"class":'stats-reference-link-viewArticle'})
CrossRef=bs_driver.find_all("a",{"class":'stats-reference-link-crossRef'})
View_Article_list=[]
CrossRef_list=[]

for article in View_Article:
    if article:
        View_Article_list.append(article.get("href"))
        
View_Article_list=['https://ieeexplore.ieee.org'+link for link in View_Article_list]

for article in CrossRef:
    if article:
        CrossRef_list.append(article.get("href"))

print(View_Article_list,CrossRef_list)

['https://ieeexplore.ieee.org/document/526899', 'https://ieeexplore.ieee.org/document/6747980', 'https://ieeexplore.ieee.org/document/1143830', 'https://ieeexplore.ieee.org/document/1164935', 'https://ieeexplore.ieee.org/document/398717', 'https://ieeexplore.ieee.org/document/1164776', 'https://ieeexplore.ieee.org/document/1573', 'https://ieeexplore.ieee.org/document/80789', 'https://ieeexplore.ieee.org/document/928686', 'https://ieeexplore.ieee.org/document/782181', 'https://ieeexplore.ieee.org/document/4545252', 'https://ieeexplore.ieee.org/document/382403', 'https://ieeexplore.ieee.org/document/267001', 'https://ieeexplore.ieee.org/document/45540', 'https://ieeexplore.ieee.org/document/388855', 'https://ieeexplore.ieee.org/document/6559349'] ['https://doi.org/10.1049/ip-f-2.1993.0002', 'https://doi.org/10.1016/j.sigpro.2009.03.033', 'https://doi.org/10.1016/j.sigpro.2014.03.002', 'https://doi.org/10.1016/j.aeue.2012.08.006']


获取期刊名

In [46]:
Journal_Title=bs_driver.find('strong', text='Published in: ').find_next_sibling().get_text()
print(Journal_Title)

'''impact_factor_list={'IEEE Transactions on Wireless Communications':10}
if Journal_Title in impact_factor_list:
    impact_factor=str(impact_factor_list[Journal_Title])
else:
    impact_factor='null'

print(impact_factor)'''

2016 Fourth International Conference on Ubiquitous Positioning, Indoor Navigation and Location Based Services (UPINLBS)


"impact_factor_list={'IEEE Transactions on Wireless Communications':10}\nif Journal_Title in impact_factor_list:\n    impact_factor=str(impact_factor_list[Journal_Title])\nelse:\n    impact_factor='null'\n\nprint(impact_factor)"

获取DOI

In [35]:
doi=bs_driver.find('strong', text='DOI: ').find_next_sibling().get_text()
Doi_link='https://doi.org/'+doi
print(Doi_link)

https://doi.org/10.1109/TWC.2009.070566


获取影响因子

In [30]:
print(bs_driver.find(href='/browse/conferences/title/')) #会议 
print(bs_driver.find({'href':'/browse/conferences/title/'}))   #TODO 为什么和上面输出不一样
print(bs_driver.find(href="/browse/periodicals/title/")) #期刊和杂志

#print(bs_driver.find(text='Journals & Magazines').parent) #这是不行的,他们都有这个,无法区分


None
None
<a _ngcontent-ijv-c160="" href="/browse/periodicals/title/">Journals &amp; Magazines</a>


In [31]:
Journal_index=False

if bs_driver.find(href="/browse/periodicals/title/"):
    
    Journal_index=True 
    print('this is a Journals or Magazines paper')

    journal_link='https://ieeexplore.ieee.org'+bs_driver.find('strong', text='Published in: ').find_next_sibling().get('href')
    print(journal_link)

    driver = webdriver.Chrome(executable_path=".\\chromedriver.exe")
    driver.get(journal_link)
    time.sleep(5)
    journal_html = driver.page_source
    driver.quit()

    f=open(".\\html\\get detail test3.html","w")
    f.write(journal_html)
    f.close()

    bs_temp=BeautifulSoup(journal_html)
    impact_factor=bs_temp.find('a',{'class':'stats-jhp-impact-factor'}).find('span',{'class':'text-md-md-lh'}).get_text()
    
else :
    impact_factor=''

print(impact_factor)

this is a Journals or Magazines paper
https://ieeexplore.ieee.org/xpl/RecentIssue.jsp?punumber=7693


C:\Users\Anable\AppData\Local\Temp\ipykernel_3992\1482078678.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=".\\chromedriver.exe")


8.346


获取日期

In [59]:
if not Journal_index:
    Date=str(bs_driver.find('strong', text='Date of Conference: ').next_sibling) # type: ignore
else:
    Date=str(bs_driver.find('strong', text='Date of Publication:').next_sibling) # type: ignore

print(Date)


 21 July 2009 


获取标题

In [33]:
print(bs.find('title').find_next_sibling().get('content')) # type: ignore #这里用.next_sibling会出错,因为两个元素间有空格

A new compact multichannel receiver for underwater wireless communication networks


获取文档PDF

In [4]:
doi='10.1109/TWC.2009.070566'
scihub_link='https://sci-hub.se/'+doi
pdf_html=urlopen(scihub_link).read()
bs_pdf=BeautifulSoup(pdf_html)

In [8]:
f=open('pdf.html','wb')
f.write(pdf_html)
f.close()

In [5]:
pdf_link='https:'+bs_pdf.find('embed',{'type':"application/pdf"}).get('src')
print(pdf_link)

https://zero.sci-hub.se/3452/c5f527d3fa8cc050a5e45aa2735b02ba/abdi2009.pdf#navpanes=0&view=FitH


In [7]:
import requests
import os

response = requests.get(pdf_link)

In [10]:
file_name = os.path.basename(pdf_link)+'.pdf'
with open('.\\pdf\\'+file_name, 'wb') as f:
    f.write(response.content)